Clone the Channelwise-Saab-Transform gitrepo to google colab

In [ ]:
!git clone https://github.com/USC-MCL/Channelwise-Saab-Transform.git

In [ ]:
!pip install numpy scikit-image numba scikit-learn xgboost tensorflow


In [ ]:
!git clone https://github.com/USC-MCL/Channelwise-Saab-Transform.git
import os
os.chdir('Channelwise-Saab-Transform')

In [ ]:
!pip install pixelhop


Modifying main.py

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist,fashion_mnist
from skimage.util import view_as_windows
from pixelhop import Pixelhop
from skimage.measure import block_reduce
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import warnings, gc
warnings.simplefilter(action='ignore', category=FutureWarning)
import time

In [ ]:
np.random.seed(1)

# Preprocess
N_Train_Reduced = 10000    # 10000
N_Train_Full = 60000     #60000
N_Test = 10000            # 10000

BS = 2000

In [ ]:
def shuffle_data(X, y):
    shuffle_idx = np.random.permutation(y.size)
    X = X[shuffle_idx]
    y = y[shuffle_idx]
    return X, y
def select_balanced_subset(images, labels, use_num_images):
    '''
    select equal number of images from each classes
    '''
    num_total, H, W, C = images.shape
    num_class = np.unique(labels).size
    num_per_class = int(use_num_images / num_class)

    # Shuffle
    images, labels = shuffle_data(images, labels)

    selected_images = np.zeros((use_num_images, H, W, C))
    selected_labels = np.zeros(use_num_images)

    for i in range(num_class):
        selected_images[i * num_per_class:(i + 1) * num_per_class] = images[labels == i][:num_per_class]
        selected_labels[i * num_per_class:(i + 1) * num_per_class] = np.ones((num_per_class)) * i

    # Shuffle again
    selected_images, selected_labels = shuffle_data(selected_images, selected_labels)

    return selected_images, selected_labels

def Shrink(X, shrinkArg):

    pool = shrinkArg['pool']
    poolh, poolw = 2, 2
    if pool:
        X = block_reduce(X, block_size=(1, poolh, poolw, 1), func=np.max)


    win = shrinkArg['win']
    stride = shrinkArg['stride']
    pad = shrinkArg['pad']

    if pad > 0:
        X = np.pad(X, ((0,0), (pad,pad), (pad,pad), (0,0)), mode='reflect')

    X = view_as_windows(X, (1, win, win, X.shape[-1]), step=(1, stride, stride, X.shape[-1]))
    shrink_arg =X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)

    return shrink_arg

def get_feat(X, num_layers=3):
    output = p2.transform_singleHop(X,layer=0)
    if num_layers>1:
        for i in range(num_layers-1):
            output = p2.transform_singleHop(output, layer=i+1)
    return output
  def Concat(X, concatArg):
    return X


In [ ]:
warnings.filterwarnings("ignore")
start_time = time.time()
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.astype('float32')[..., np.newaxis] / 255.0, x_test.astype('float32')[..., np.newaxis] / 255.0
y_train, y_test = y_train.astype('int'), y_test.astype('int')
x_train_reduced, y_train_reduced = select_balanced_subset(x_train, y_train, N_Train_Reduced)

print('Running Module 1...')
SaabArgs = [{'num_AC_kernels': -1, 'needBias': False, 'cw': False}, {'num_AC_kernels': -1, 'needBias': True, 'cw': True}, {'num_AC_kernels': -1, 'needBias': True, 'cw': True}]
shrinkArgs = [{'func': Shrink, 'win': 5, 'stride': 1, 'pad': 2, 'pool': False}, {'func': Shrink, 'win': 5, 'stride': 1, 'pad': 0, 'pool': True}, {'func': Shrink, 'win': 5, 'stride': 1, 'pad': 0, 'pool': True}]

concatArg = {'func': Concat}

#module 1
p2 = Pixelhop(depth=3, TH1=0.001, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
p2.fit(x_train_reduced)

#Module 2
train_hop3_feats = get_feat(x_train)
test_hop3_feats = get_feat(x_test)
STD = np.std(train_hop3_feats, axis=0, keepdims=True)
train_hop3_feats /= STD
test_hop3_feats /= STD

train_hop3_reshaped = train_hop3_feats.reshape(train_hop3_feats.shape[0], -1)
test_hop3_reshaped = test_hop3_feats.reshape(test_hop3_feats.shape[0], -1)

# Module 3
clf = xgb.XGBClassifier(n_jobs=-1, objective='multi:softprob', max_depth=6, n_estimators=100, min_child_weight=5, gamma=5, subsample=0.8, learning_rate=0.1, nthread=8, colsample_bytree=1.0)
clf.fit(train_hop3_reshaped, y_train)

pred_train = clf.predict(train_hop3_reshaped)
pred_test = clf.predict(test_hop3_reshaped)
train_accuracy = accuracy_score(y_train, pred_train)
test_accuracy = accuracy_score(y_test, pred_test)

end_time = time.time()
training_time = end_time - start_time

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("Training Time:", training_time, "seconds")

feats_K1 = get_feat(x_train,num_layers=0)
feats_K2 = get_feat(x_train,num_layers=2)
feats_K3 = get_feat(x_train,num_layers=3)
K1 = feats_K1.shape[-1]
K2 = feats_K2.shape[-1]
K3 = feats_K3.shape[-1]
print("K1 Features:", K1)
print("K2 Features:", K2)
print("K3 Features:", K3)

In [ ]:

th1_values = [ 0.001,0.005, 0.01, 0.05, 0.1]
test_accuracies = [ 96.63,96.28, 96.14, 93.12, 90.87]

plt.figure(figsize=(10, 5))
plt.plot(th1_values, test_accuracies, marker='o', linestyle='-', color='g')
plt.title('TH1 vs. Test Accuracy')
plt.xlabel('TH1 Values')
plt.ylabel('Test Accuracy (%)')
plt.grid(True)
plt.show()

In [ ]:
cm = confusion_matrix(y_test, pred_test)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

class_accuracy = cm.diagonal() / cm.sum(axis=1)
lowest_error_class = np.argmin(class_accuracy)
highest_error_class = np.argmax(class_accuracy)

print(f"Class with the lowest error rate: {lowest_error_class} (Error Rate: {1 - class_accuracy[lowest_error_class]:.2f})")
print(f"Class with the highest error rate: {highest_error_class} (Error Rate: {1 - class_accuracy[highest_error_class]:.2f})")
